In [ ]:
!pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-4e5naycs
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-4e5naycs
  Resolved https://github.com/openai/whisper.git to commit 517a43ecd132a2089d85f4ebc044728a71d49f6e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━

In [ ]:
!pip install librosa
!pip install pytest-warnings

In [ ]:
!pip install langchain langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 33.2 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.4 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.16 which is incompatible.


In [ ]:
!pip install audioop-lts

In [ ]:
!pip install pydub

## Main code Modified (11/3/2025)

In [ ]:
import librosa
import numpy as np
from pydub import AudioSegment
from pydub.silence import detect_silence
from pydub.utils import db_to_float
import whisper
import warnings
import wave
import audioop
import json
# import psutil
# from flask import Flask, jsonify, request
import os
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain_google_genai import ChatGoogleGenerativeAI


# app = Flask(__name__)

google_api_key = ''

llm = ChatGoogleGenerativeAI(model='gemini-1.5-pro-latest', temperature=0.6, google_api_key=google_api_key)

memory = ConversationBufferMemory()
conversation_chain = ConversationChain(llm=llm, memory=memory)

whisper_model = whisper.load_model("small")

# Load audio file
def load_audio(file_path):
    audio, sr = librosa.load(file_path, sr=None)
    return audio, sr

# Convert NumPy array to AudioSegment
def numpy_to_audiosegment(audio, sr):
    audio = (audio * (2**15 - 1)).astype(np.int16)  # Convert to 16-bit PCM format
    audio_segment = AudioSegment(
        audio.tobytes(),
        frame_rate=sr,
        sample_width=audio.dtype.itemsize,
        channels=1
    )
    return audio_segment

# Analyze pitch and tone
def analyze_pitch_and_tone(audio, sr):
    pitches, magnitudes = librosa.core.piptrack(y=audio, sr=sr)
    pitch_mean = np.mean(pitches[pitches > 0])  # Ignore zero values
    pitch_std = np.std(pitches[pitches > 0])
    return pitch_mean, pitch_std

# Analyze speech rate
def analyze_speech_rate(transcript, audio_duration):
    words = transcript.split()
    word_count = len(words)
    speech_rate = word_count / audio_duration
    return speech_rate

# Detect pauses and fillers
def detect_pauses_and_fillers(audio_segment, transcript):
    # Detect silence (pauses)
    silence_threshold = -40  # dB
    min_silence_len = 500  # ms
    pauses = detect_silence(audio_segment, min_silence_len, silence_threshold, 1)

    # Count fillers ("um", "uh")
    fillers = ["um", "uh"]
    filler_count = sum(transcript.lower().count(filler) for filler in fillers)

    return pauses, filler_count

# Analyze volume and clarity
def analyze_volume_and_clarity(audio):
    rms = librosa.feature.rms(y=audio)[0]
    volume_mean = np.mean(rms)
    volume_std = np.std(rms)
    return volume_mean, volume_std

# Audio to text using Whisper
def transcribe_audio_whisper(file_path):
    warnings.filterwarnings("ignore", message="You are using `torch.load` with `weights_only=False`")
    print("Transcribing audio with Whisper...")
    result = whisper_model.transcribe(file_path, fp16=False)
    return result["text"]

def analyze_audio(file_path):
    audio, sr = load_audio(file_path)
    audio_duration = librosa.get_duration(y=audio, sr=sr)

    # Convert audio to AudioSegment
    audio_segment = numpy_to_audiosegment(audio, sr)

    # Transcribe audio with Whisper
    transcript = transcribe_audio_whisper(file_path)
    if not transcript:
        print("No transcript available for analysis.")
        return

    # Analyze features
    pitch_mean, pitch_std = analyze_pitch_and_tone(audio, sr)
    speech_rate = analyze_speech_rate(transcript, audio_duration)
    pauses, filler_count = detect_pauses_and_fillers(audio_segment, transcript)
    volume_mean, volume_std = analyze_volume_and_clarity(audio)

    # Analyze basic properties of the new audio file
    new_audio_properties = {}
    try:
        with wave.open(file_path, 'r') as audio_:
            # Extract basic audio properties
            new_audio_properties['frame_rate'] = audio_.getframerate()
            new_audio_properties['n_frames'] = audio_.getnframes()
            new_audio_properties['duration'] = audio_.getnframes() / audio_.getframerate()

            # Compute average loudness (RMS) and silence ratio
            frames = audio_.readframes(audio_.getnframes())
            rms = audioop.rms(frames, audio_.getsampwidth())  # Root mean square of the audio signal
            silent_frames = sum(
                audioop.rms(frames[i:i + audio_.getsampwidth()], audio_.getsampwidth()) < 1000
                for i in range(0, len(frames), audio_.getsampwidth())
            )
            silence_ratio = silent_frames / len(frames)
    except Exception as e:
        new_audio_properties['error'] = str(e)

    # Package detailed results for the new audio file
    new_audio_properties.update({
        "average_loudness": rms,
        "silence_ratio": silence_ratio,
        "speech_detected": silence_ratio < 0.5,  # Assumes speech if less than 50% silence
    })

    # Output the analysis results
    print(new_audio_properties)

    scores = []

    # Pauses condition
    if 10 <= len(pauses) <= 20:
        scores.append('Average')
    elif 1 <= len(pauses) <= 10:
        scores.append('Good')
    elif len(pauses) == 0:
        scores.append('Excellent')
    elif len(pauses) > 20:
        scores.append('Bad')

    # Speech rate condition
    if speech_rate <= 1.25:
        scores.append('Bad')
    elif 1.25 <= speech_rate <= 2.05:
        scores.append('Average')
    elif 2.05 <= speech_rate <= 3.0:
        scores.append('Good')
    else:
        scores.append('Excellent')

    # Silence Ratio
    if silence_ratio > 0.45:
        scores.append('Bad')
    elif 0 <= silence_ratio <= 0.20:
        scores.append('Excellent')
    elif 0.20 < silence_ratio <= 0.35:
        scores.append('Good')
    elif 0.35 < silence_ratio <= 0.45:
        scores.append('Average')

    # Pitch mean
    if pitch_mean <= 732:
        scores.append('Bad')
    elif 732 < pitch_mean <= 900:
        scores.append('Average')
    elif pitch_mean > 900:
        scores.append('Good')

    # Pitch Std
    if pitch_std <= 723:
        scores.append('Bad')
    elif 723 < pitch_std <= 900:
        scores.append('Average')
    elif pitch_std > 900:
        scores.append('Good')

    print(scores)

    if pitch_mean > 1350 or rms > 2800:
        pitch_mean = pitch_mean / 1.25
        rms = rms / 1.75

        Average_positive = [pitch_mean, pitch_std, speech_rate, rms]
        Average_negative = [len(pauses), (silence_ratio * 10)]

        Scaled_positive = sum(Average_positive) / 4
        Scaled_negative = sum(Average_negative) / 2

        Scaled_positive_1000 = Scaled_positive / 1000
        Scaled_negative_100 = Scaled_negative / 100

        final_conversion_p = Scaled_positive_1000 * 0.25
        final_conversion_n = Scaled_negative_100 * 0.50

        Result = final_conversion_p - final_conversion_n
        print(final_conversion_p, '\n', final_conversion_n)

        Result_new = Result * 2.5
        final_result = Result_new * 100
        # print(f'Your final Audio Score is {final_result}')

    else:
        Average_positive = [pitch_mean, pitch_std, speech_rate]
        Average_negative = [len(pauses), (silence_ratio * 10)]

        Scaled_positive = sum(Average_positive) / 3
        Scaled_negative = sum(Average_negative) / 2

        Scaled_positive_1000 = Scaled_positive / 1000
        Scaled_negative_100 = Scaled_negative / 100

        final_conversion_p = Scaled_positive_1000 * 0.25
        final_conversion_n = Scaled_negative_100 * 0.50

        Result = final_conversion_p - final_conversion_n
        Result_new = Result * 2.5
        final_result = Result_new * 100



    audio_score = []

    if scores.count('Bad') >= 2:
        final_result_new = final_result / 2
        audio_score.append(final_result_new)
        print(f'Your final Audio Score is {final_result_new}')
    elif scores.count('Bad') == 1:
        decrease = final_result * (1 / 4)
        final_result_new = final_result - decrease
        audio_score.append(final_result_new)
        print(f'Your final Audio Score is {final_result_new}')
    else:
        print(f'Your final Audio Score is {final_result}')
        audio_score.append(final_result)

      # Define the prompt for analysis
    analysis_prompt = f""" {transcript} analysis the text based on grammar mistakes, fluency, clarity , professional tone and  overall rate it and rate out of 100 this is the text of interviewer analysis it completely Provide me only the scores don't give unnecessary text also detect irrelevant words means(out of context words or topic like here is candidate self introduction text if find irrelevant topic than self introduction) and give me score if there are out of context words and drop overall score if there is irrelvancy return only overall score not return any other score give me final score by decreasing the (give me irrelevancy score in positive number only don't give negative number return in json format)
    """
    print(transcript)

    def chat_with_bot(user_input):
        formatted_prompt = analysis_prompt.format(text=user_input)
        response = conversation_chain.run(input=formatted_prompt)
        return response

    response = chat_with_bot(user_input=analysis_prompt)

    start = response.replace("```json", "")
    end = start.replace("```", "")
    end1 = json.loads(end)
    print(end1)
    print(end1["overall"] - end1["irrelevancy"])
    text_final_score = end1["overall"] - end1["irrelevancy"]
    audio_final_score = audio_score[0]

    overall_final_score = (text_final_score + audio_final_score) / 2
    print(f"overall final score is {overall_final_score}")

if __name__ == "__main__":
    audio_file = "/content/silent audio1.wav"
    analyze_audio(audio_file)


In [ ]:
import librosa
import numpy as np
from pydub import AudioSegment
from pydub.silence import detect_silence
from pydub.utils import db_to_float
import whisper
import warnings
import wave
import audioop

# Load Whisper medium model
whisper_model = whisper.load_model("small")

# Load audio file
def load_audio(file_path):
    audio, sr = librosa.load(file_path, sr=None)
    return audio, sr

# Convert NumPy array to AudioSegment
def numpy_to_audiosegment(audio, sr):
    audio = (audio * (2**15 - 1)).astype(np.int16)  # Convert to 16-bit PCM format
    audio_segment = AudioSegment(
        audio.tobytes(),
        frame_rate=sr,
        sample_width=audio.dtype.itemsize,
        channels=1
    )
    return audio_segment

# Analyze pitch and tone
def analyze_pitch_and_tone(audio, sr):
    pitches, magnitudes = librosa.core.piptrack(y=audio, sr=sr)
    pitch_mean = np.mean(pitches[pitches > 0])  # Ignore zero values
    pitch_std = np.std(pitches[pitches > 0])
    return pitch_mean, pitch_std

# Analyze speech rate
def analyze_speech_rate(transcript, audio_duration):
    words = transcript.split()
    word_count = len(words)
    speech_rate = word_count / audio_duration
    return speech_rate

# Detect pauses and fillers
def detect_pauses_and_fillers(audio_segment, transcript):
    # Detect silence (pauses)
    silence_threshold = -40  # dB
    min_silence_len = 500  # ms
    pauses = detect_silence(audio_segment, min_silence_len, silence_threshold, 1)

    # Count fillers ("um", "uh")
    fillers = ["um", "uh"]
    filler_count = sum(transcript.lower().count(filler) for filler in fillers)

    return pauses, filler_count

# Analyze volume and clarity
def analyze_volume_and_clarity(audio):
    rms = librosa.feature.rms(y=audio)[0]
    volume_mean = np.mean(rms)
    volume_std = np.std(rms)
    return volume_mean, volume_std

#Audio to text using Whisper
def transcribe_audio_whisper(file_path):
    warnings.filterwarnings("ignore", message="You are using `torch.load` with `weights_only=False`")
    print("Transcribing audio with Whisper...")
    result = whisper_model.transcribe(file_path, fp16=False)
    return result["text"]


def analyze_audio(file_path):
    audio, sr = load_audio(file_path)
    audio_duration = librosa.get_duration(y=audio, sr=sr)

    # Convert audio to AudioSegment
    audio_segment = numpy_to_audiosegment(audio, sr)

    # Transcribe audio with Whisper
    transcript = transcribe_audio_whisper(file_path)
    if not transcript:
        print("No transcript available for analysis.")
        return

    # Analyze features
    pitch_mean, pitch_std = analyze_pitch_and_tone(audio, sr)
    speech_rate = analyze_speech_rate(transcript, audio_duration)
    pauses, filler_count = detect_pauses_and_fillers(audio_segment, transcript)
    volume_mean, volume_std = analyze_volume_and_clarity(audio)



    # Analyze basic properties of the new audio file
    new_audio_properties = {}
    try:
        with wave.open(audio_file, 'r') as audio_:
            # Extract basic audio properties
            # new_audio_properties['channels'] = audio_file.getnchannels()
            # new_audio_properties['sample_width'] = audio_file.getsampwidth()
            new_audio_properties['frame_rate'] = audio_.getframerate()
            new_audio_properties['n_frames'] = audio_.getnframes()
            new_audio_properties['duration'] = audio_.getnframes() / audio_.getframerate()

            # Compute average loudness (RMS) and silence ratio
            frames = audio_.readframes(audio_.getnframes())
            rms = audioop.rms(frames, audio_.getsampwidth())  # Root mean square of the audio signal
            silent_frames = sum(
                audioop.rms(frames[i:i + audio_.getsampwidth()], audio_.getsampwidth()) < 1000
                for i in range(0, len(frames), audio_.getsampwidth())
            )
            silence_ratio = silent_frames / len(frames)
    except Exception as e:
        new_audio_properties['error'] = str(e)

    # Package detailed results for the new audio file
    new_audio_properties.update({
        "average_loudness": rms,
        "silence_ratio": silence_ratio,
        "speech_detected": silence_ratio < 0.5,  # Assumes speech if less than 50% silence
    })

    # Output the analysis results
    print(new_audio_properties)

    # results
    print("Analysis Results:")
    print(f"Pitch Mean: {pitch_mean:.2f}, Pitch Std: {pitch_std:.2f}")
    print(f"Speech Rate: {speech_rate:.2f} words per second")
    print(f"Number of Pauses: {len(pauses)}")
    print(f"Filler Count: {filler_count}")
    print(f"Volume Mean: {volume_mean:.2f}, Volume Std: {volume_std:.2f}")
    print(f"Transcript: {transcript}")

    if pitch_mean>1350 or rms>2800:
      pitch_mean = pitch_mean/1.25
      rms = rms/1.85

      Average_positive = [pitch_mean,pitch_std,speech_rate,rms]
      Average_negative = [len(pauses),(silence_ratio*10)]


      Scaled_positive = sum(Average_positive)/4
      Scaled_negative = sum(Average_negative)/2

      Scaled_positive_1000 = Scaled_positive/1000
      Scaled_negative_100 = Scaled_negative/50


      final_conversion_p = Scaled_positive_1000*0.25
      final_conversion_n = Scaled_negative_100*0.50


      Result = final_conversion_p-final_conversion_n
      Result_new = Result*1.75
      final_result = Result_new*100
      print(f'Your final Audio Score is {final_result}')

    else:
      Average_positive = [pitch_mean,pitch_std,speech_rate,rms]
      Average_negative = [len(pauses),(silence_ratio*10)]


      Scaled_positive = sum(Average_positive)/4
      Scaled_negative = sum(Average_negative)/2

      Scaled_positive_1000 = Scaled_positive/1000
      Scaled_negative_100 = Scaled_negative/50


      final_conversion_p = Scaled_positive_1000*0.25
      final_conversion_n = Scaled_negative_100*0.50


      Result = final_conversion_p-final_conversion_n
      # Result_new = Result*2.5
      final_result = Result*100
      if final_result <=0:
        print('Your score is 10')
      else:
        print(f'Your final Audio Score is {final_result}')


    # print(f"Results:{Result*2.5}")
    # print(f)

    # print(Average_positive)

    # import os
    # from langchain.chains import ConversationChain
    # from langchain.chains.conversation.memory import ConversationBufferMemory
    # from langchain_google_genai import ChatGoogleGenerativeAI

    # # API KEY
    # google_api_key = ''

    # # Llm model google genai
    # llm = ChatGoogleGenerativeAI(model='gemini-1.5-pro-latest', temperature=0.6, google_api_key=google_api_key)

    # # Set up memory and conversation chain
    # memory = ConversationBufferMemory()
    # conversation_chain = ConversationChain(llm=llm, memory=memory)

    # # Define the prompt for analysis
    # analysis_prompt = f""" {transcript} analysis the text based on grammar mistakes, fluency, clarity , professional tone and  overall rate it and rate out of 10 this is the text of interviewer analysis it completely Provide me only the scores don't give unnecessary text

    # """


    # def chat_with_bot(user_input):
    #     # Format the analysis prompt with user input
    #     formatted_prompt = analysis_prompt.format(text=user_input)
    #     response = conversation_chain.run(input=formatted_prompt)
    #     return response

    # print(f"Your Audio Text score is \n {chat_with_bot(user_input=analysis_prompt)}")


    print(transcript)

if __name__ == "__main__":
    audio_file = "/content/kanisha_n.wav"
    analyze_audio(audio_file)

100%|███████████████████████████████████████| 461M/461M [00:05<00:00, 93.1MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

Transcribing audio with Whisper...
{'frame_rate': 48000, 'n_frames': 2022912, 'duration': 42.144, 'average_loudness': 4791, 'silence_ratio': 0.23425981950771957, 'speech_detected': True}
Analysis Results:
Pitch Mean: 1289.36, Pitch Std: 1015.02
Speech Rate: 2.56 words per second
Number of Pauses: 0
Filler Count: 0
Volume Mean: 0.09, Volume Std: 0.06
Transcript:  My name is Kaneshka. I am currently pursuing my BICOM along with my studies. I have industrial experience working at VSCO HR private limited. This role has helped me help me in develop strong communication skills, teamwork and problem-solving abilities. I am a quick learner always looking to improve my skills and take on new challenges. I am excited to apply my knowledge and experience to a new role and contribute to a team. I am confident that my strong work ethic, positive attitude and passion of learning will make me a valuable asset to any organization. I am looking forward to discussing my qualification further. Thank you.

In [ ]:
!python --version

Python 3.11.11


In [ ]:
number = 21
if 10 <= number <= 20:
    print('Average')
elif 1<= number <=10:
  print('Good')
elif number == 0:
    print('Excellent')
elif number > 20:
    print('Bad')



Bad


## Main or Updated code

In [ ]:
import librosa
import numpy as np
from pydub import AudioSegment
from pydub.silence import detect_silence
from pydub.utils import db_to_float
import whisper
import warnings
import wave
import audioop

#  Whisper medium
whisper_model = whisper.load_model("medium")

# Load audio file
def load_audio(file_path):
    audio, sr = librosa.load(file_path, sr=None)
    return audio, sr

# Convert NumPy array to AudioSegment
def numpy_to_audiosegment(audio, sr):
    audio = (audio * (2**15 - 1)).astype(np.int16)  # Convert to 16-bit PCM format
    audio_segment = AudioSegment(
        audio.tobytes(),
        frame_rate=sr,
        sample_width=audio.dtype.itemsize,
        channels=1
    )
    return audio_segment

# Analyze pitch and tone
def analyze_pitch_and_tone(audio, sr):
    pitches, magnitudes = librosa.core.piptrack(y=audio, sr=sr)
    pitch_mean = np.mean(pitches[pitches > 0])  # Ignore zero values
    pitch_std = np.std(pitches[pitches > 0])
    return pitch_mean, pitch_std

# Analyze speech rate
def analyze_speech_rate(transcript, audio_duration):
    words = transcript.split()
    word_count = len(words)
    speech_rate = word_count / audio_duration
    return speech_rate

# Detect pauses and fillers
def detect_pauses_and_fillers(audio_segment, transcript):
    # Detect silence (pauses)
    silence_threshold = -40  # dB
    min_silence_len = 500  # ms
    pauses = detect_silence(audio_segment, min_silence_len, silence_threshold, 1)

    # Count fillers ("um", "uh")
    fillers = ["um", "uh"]
    filler_count = sum(transcript.lower().count(filler) for filler in fillers)

    return pauses, filler_count

# Analyze volume and clarity
def analyze_volume_and_clarity(audio):
    rms = librosa.feature.rms(y=audio)[0]
    volume_mean = np.mean(rms)
    volume_std = np.std(rms)
    return volume_mean, volume_std

#Audio to text using Whisper
def transcribe_audio_whisper(file_path):
    warnings.filterwarnings("ignore", message="You are using `torch.load` with `weights_only=False`")
    print("Transcribing audio with Whisper...")
    result = whisper_model.transcribe(file_path, fp16=False)
    return result["text"]


def analyze_audio(file_path):
    audio, sr = load_audio(file_path)
    audio_duration = librosa.get_duration(y=audio, sr=sr)

    # Convert audio to AudioSegment
    audio_segment = numpy_to_audiosegment(audio, sr)

    # Transcribe audio with Whisper
    transcript = transcribe_audio_whisper(file_path)
    if not transcript:
        print("No transcript available for analysis.")
        return

    # Analyze features
    pitch_mean, pitch_std = analyze_pitch_and_tone(audio, sr)
    speech_rate = analyze_speech_rate(transcript, audio_duration)
    pauses, filler_count = detect_pauses_and_fillers(audio_segment, transcript)
    volume_mean, volume_std = analyze_volume_and_clarity(audio)



    # Analyze basic properties of the new audio file
    new_audio_properties = {}
    try:
        with wave.open(audio_file, 'r') as audio_:
            # Extract basic audio properties
            # new_audio_properties['channels'] = audio_file.getnchannels()
            # new_audio_properties['sample_width'] = audio_file.getsampwidth()
            new_audio_properties['frame_rate'] = audio_.getframerate()
            new_audio_properties['n_frames'] = audio_.getnframes()
            new_audio_properties['duration'] = audio_.getnframes() / audio_.getframerate()

            # Compute average loudness (RMS) and silence ratio
            frames = audio_.readframes(audio_.getnframes())
            rms = audioop.rms(frames, audio_.getsampwidth())  # Root mean square of the audio signal
            silent_frames = sum(
                audioop.rms(frames[i:i + audio_.getsampwidth()], audio_.getsampwidth()) < 1000
                for i in range(0, len(frames), audio_.getsampwidth())
            )
            silence_ratio = silent_frames / len(frames)
    except Exception as e:
        new_audio_properties['error'] = str(e)

    # Package detailed results for the new audio file
    new_audio_properties.update({
        "average_loudness": rms,
        "silence_ratio": silence_ratio,
        "speech_detected": silence_ratio < 0.5,  # Assumes speech if less than 50% silence
    })

    # Output the analysis results
    # print(new_audio_properties)

    # results
    # print("Analysis Results:")
    # print(f"Pitch Mean: {pitch_mean:.2f}, Pitch Std: {pitch_std:.2f}")
    # print(f"Speech Rate: {speech_rate:.2f} words per second")
    # print(f"Number of Pauses: {len(pauses)}")
    # print(f"Filler Count: {filler_count}")
    # print(f"Volume Mean: {volume_mean:.2f}, Volume Std: {volume_std:.2f}")
    # print(f"Transcript: {transcript}")

    scores = []

    # Pauses conditon
    print('Pauses Score')
    if 10 <= len(pauses) <= 20:
        scores.append('Average')
        # print('Average')
    elif 1<=len(pauses)<=10:
      scores.append('Good')
      # print('GOOD')
    elif len(pauses) == 0:
        scores.append('Excellent')
        # print('Excellent')
    elif len(pauses) > 20:
        scores.append('Bad')
        # print('Bad')

    # Speech rate condition
    print('Speech rate Score')
    if speech_rate<=1.25:
      scores.append('Bad')
      # print('Bad')
    elif 1.25<=speech_rate<=2.05:
      scores.append('Average')
      # print('Average')
    elif 2.05<=speech_rate<=3.0:
      scores.append('Good')
      # print('Good')
    else:
      scores.append('Excellent')
      # print('Excellent')

    # Silence_Ratio
    print('Silence Ratio Score')
    if silence_ratio>0.45:
      scores.append('Bad')
      # print('Bad')
    elif 0<=silence_ratio<=0.20:
      scores.append('Excellent')
      # print('Excellent')
    elif 0.20<silence_ratio<=0.35:
      scores.append('Good')
      # print('Good')
    elif 0.35<silence_ratio<=0.45:
      scores.append('Average')
      # print('Average')

    # Pitch mean
    print('Pitch mean Score')
    if pitch_mean<=732:
      scores.append('Bad')
      # print('Bad')
    elif 732<pitch_mean<=900:
      scores.append('Average')
      # print('Average')
    elif pitch_mean>900:
      scores.append('Good')
      # print('Good')

    # Pitch Std
    print('Pitch Std score')
    if pitch_std<=723:
      scores.append('Bad')
      # print('Bad')
    elif 723<pitch_std<=900:
      scores.append('Average')
      # print('Average')
    elif pitch_std>900:
      scores.append('Good')
      # print('Good')


    if pitch_mean>1350 or rms>2800:
      pitch_mean = pitch_mean/1.25
      rms = rms/1.75

      Average_positive = [pitch_mean,pitch_std,speech_rate,rms]
      Average_negative = [len(pauses),(silence_ratio*10)]


      Scaled_positive = sum(Average_positive)/4
      Scaled_negative = sum(Average_negative)/2

      Scaled_positive_1000 = Scaled_positive/1000
      Scaled_negative_100 = Scaled_negative/100


      final_conversion_p = Scaled_positive_1000*0.25
      final_conversion_n = Scaled_negative_100*0.50


      Result = final_conversion_p-final_conversion_n
      print(final_conversion_p,'\n',final_conversion_n)
      # Result = final_conversion_p
      Result_new = Result*2.5
      final_result = Result_new*100
      print(f'Your final Audio Score is {final_result}')

    else:
        Average_positive = [pitch_mean,pitch_std,speech_rate]
        Average_negative = [len(pauses),(silence_ratio*10)]


        Scaled_positive = sum(Average_positive)/3
        Scaled_negative = sum(Average_negative)/2

        Scaled_positive_1000 = Scaled_positive/1000
        Scaled_negative_100 = Scaled_negative/100


        final_conversion_p = Scaled_positive_1000*0.25
        final_conversion_n = Scaled_negative_100*0.50


        Result = final_conversion_p-final_conversion_n
        Result_new = Result*2.5
        final_result = Result_new*100
        # print(f'Your final Audio Score is {final_result}')


        # print(scores)

        if scores.count('Bad')>=2:
          final_result_new = final_result/2
          print("Your final audio score is ", final_result_new)
        elif scores.count('Bad')==1:
          decrease = final_result*(1/4)
          final_result_new = final_result-decrease
          print('Your final audio score is ',final_result_new)
        else:
          print('Your final audio score is ',final_result)
    # print(f"Results:{Result*2.5}")
    # print(f)

    # print(Average_positive)

    import os
    from langchain.chains import ConversationChain
    from langchain.chains.conversation.memory import ConversationBufferMemory
    from langchain_google_genai import ChatGoogleGenerativeAI


    google_api_key = ''

    # Llm model google genai
    llm = ChatGoogleGenerativeAI(model='gemini-1.5-pro-latest', temperature=0.6, google_api_key=google_api_key)

    # Set up memory and conversation chain
    memory = ConversationBufferMemory()
    conversation_chain = ConversationChain(llm=llm, memory=memory)

    overall_text_score = ''
    # Define the prompt for analysis
    analysis_prompt = f""" {transcript} analysis the text based on grammar mistakes, fluency, clarity , professional tone and  overall rate it and rate out of 10 this is the text of interviewer analysis it completely Provide me only the scores don't give unnecessary text also detect irrelevant words and give me score if there are out of context words and drop overall score if there is irrelvancy store overall score in this variable{overall_text_score}

    """
    print(overall_text_score)
    def chat_with_bot(user_input):
        # Format the analysis prompt with user input
        formatted_prompt = analysis_prompt.format(text=user_input)
        response = conversation_chain.run(input=formatted_prompt)
        return response



    output_text = chat_with_bot(user_input=analysis_prompt)
    print(output_text)
    print(f"Your Audio Text score is \n {chat_with_bot(user_input=analysis_prompt)}")

    print(overall_text_score,'overall_text score')
    print(transcript)



audio_file = "/content/kanisha_n.wav"
analyze_audio(audio_file)
# if __name__ == "__main__":
#     audio_file = "/content/kanisha_n.wav"
#     analyze_audio(audio_file)

Transcribing audio with Whisper...
Pauses Score
Speech rate Score
Silence Ratio Score
Pitch mean Score
Pitch Std score
0.29917248240424443 
 0.00585649548769299
Your final Audio Score is 73.32899672913786

Grammar: 7/10
Fluency: 6/10
Clarity: 8/10
Professional Tone: 9/10
Irrelevant Words: 9/10 (Minor issue with "along with my studies" - redundant after "pursuing my B.Com")

Overall: 7.8/10 
Your Audio Text score is 
 Grammar: 7/10
Fluency: 6/10
Clarity: 8/10
Professional Tone: 9/10
Irrelevant Words: 9/10

Overall: 7.8/10
 overall_text score
 My name is Kanishka. I am currently pursuing my B.Com along with my studies. I have industrial experience working at Wisco HR Private Limited. This role has helped me in developing strong communication skills, teamwork and problem solving abilities. I am a quick learner, always looking to improve my skills and take on new challenges. I am excited to apply my knowledge and experience to a new role and contribute to a team. I am confident that my str

In [ ]:

import os
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain_google_genai import ChatGoogleGenerativeAI


google_api_key = ''

# Llm model google genai
llm = ChatGoogleGenerativeAI(model='gemini-1.5-pro-latest', temperature=0.6, google_api_key=google_api_key)

# Set up memory and conversation chain
memory = ConversationBufferMemory()
conversation_chain = ConversationChain(llm=llm, memory=memory)

transcript = 'My name is Kanishka. I am currently pursuing my B.Com along with my studies. I have industrial experience working at Wisco HR Private Limited. This role has helped me in developing strong communication skills, teamwork and problem solving abilities. I am a quick learner, always looking to improve my skills and take on new challenges. I am excited to apply my knowledge and experience to a new role and contribute to a team. I am confident that my strong work ethic, positive attitude and passion of learning will make me a valuable asset to any organization. I am looking forward to discussing my qualification further. Thank you'
transcript2 = 'My name is Kanishka. I am currently pursuing my B.Com along with my studies. I have industrial experience working at Wisco HR Private Limited. This role has helped me in developing strong communication skills, teamwork and problem solving abilities. I love playing cricket with my boyfriend and watching tv shows with family I enjoyed traveling and dancing. I am a quick learner, always looking to improve my skills and take on new challenges. I am excited to apply my knowledge and experience to a new role and contribute to a team. I am confident that my strong work ethic, positive attitude and passion of learning will make me a valuable asset to any organization. I am looking forward to discussing my qualification further. Thank you'
# Define the prompt for analysis
analysis_prompt = f""" {transcript2}
.

"""
# print(overall_text_score)
def chat_with_bot(user_input):
    # Format the analysis prompt with user input
    formatted_prompt = analysis_prompt.format(text=user_input)
    response = conversation_chain.run(input=formatted_prompt)
    return response





output_text = chat_with_bot(user_input=analysis_prompt)
output = output_text.split(' ')
final_output = output[-1::]
print(output_text)
print(final_output)

# print(output_text)
# print(f"Your Audio Text score is \n {chat_with_bot(user_input=analysis_prompt)}")

# # print(overall_text_score,'overall_text score')
# print(transcript)


Grammar: 6/10
Fluency: 7/10
Clarity: 7/10
Professional Tone: 6/10
Overall: 6.5/10
Irrelevance: 8/10 (Mentioning boyfriend and watching TV with family is irrelevant in this context)  This lowers the overall score due to the inclusion of personal, irrelevant details.

**Final Overall (adjusted for irrelevance): 5.5/10**
['5.5/10**']


In [ ]:
score = output_text.split()

In [ ]:
score[-1]

'5.5/10'

In [ ]:
values = ['Good', 'Bad', 'Bad', 'Good', 'Good','Good']

if values.count('Bad')==2:
  print('hii')
else:
  print('no')

hii


 My name is Kaneshka. I am currently pursuing my BICOM along with my studies. I have industrial experience working at VSCO HR private limited. This role has helped me help me in develop strong communication skills, teamwork and problem-solving abilities. I am a quick learner always looking to improve my skills and take on new challenges. I am excited to apply my knowledge and experience to a new role and contribute to a team. I am confident that my strong work ethic, positive attitude and passion of learning will make me a valuable asset to any organization. I am looking forward to discussing my qualification further. Thank you.

### ritik
Transcribing audio with Whisper...
{'frame_rate': 48000, 'n_frames': 3280896, 'duration': 68.352, 'average_loudness': 1594, 'silence_ratio': 0.3440546423903714, 'speech_detected': True}
Analysis Results:
Pitch Mean: 1063.82, Pitch Std: 984.70
Speech Rate: 2.02 words per second
Number of Pauses: 22
Filler Count: 0
Volume Mean: 0.04, Volume Std: 0.03
Your final Audio Score is 25.14521660132057

Your final Audio Score is 25.14521660132057
Your Audio Text score is
 Grammar: 6/10
Fluency: 7/10
Clarity: 7/10
Professional Tone: 6/10
Overall: 6.5/10

atul
Transcribing audio with Whisper...
{'frame_rate': 48000, 'n_frames': 5020416, 'duration': 104.592, 'average_loudness': 2351, 'silence_ratio': 0.3117542450665443, 'speech_detected': True}
Analysis Results:
Pitch Mean: 1134.37, Pitch Std: 991.77
Speech Rate: 1.49 words per second
Number of Pauses: 18
Filler Count: 1
Volume Mean: 0.06, Volume Std: 0.05


Your final Audio Score is 43.581676155780094
  response = conversation_chain.run(input=formatted_prompt)
Your Audio Text score is
 Grammar: 4/10
Fluency: 3/10
Clarity: 6/10
Professional Tone: 4/10
Overall: 4/10

### hema
Transcribing audio with Whisper...
{'frame_rate': 44100, 'n_frames': 2850048, 'duration': 64.6269387755102, 'average_loudness': 3719, 'silence_ratio': 0.27156191755366926, 'speech_detected': True}
Analysis Results:
Pitch Mean: 1525.97, Pitch Std: 1129.75
Speech Rate: 1.76 words per second
Number of Pauses: 16
Filler Count: 0
Volume Mean: 0.08, Volume Std: 0.08


Your final Audio Score is 76.23804714535592
Your Audio Text score is
 Grammar: 6/10
Fluency: 7/10
Clarity: 7/10
Professional Tone: 6/10
Overall: 6.5/10



### kanishka

Transcribing audio with Whisper...
{'frame_rate': 48000, 'n_frames': 2022912, 'duration': 42.144, 'average_loudness': 4791, 'silence_ratio': 0.23425981950771957, 'speech_detected': True}
Analysis Results:
Pitch Mean: 1289.36, Pitch Std: 1015.02
Speech Rate: 2.56 words per second
Number of Pauses: 0
Filler Count: 0
Volume Mean: 0.09, Volume Std: 0.06
Your final Audio Score is 107.97709086067435
Your Audio Text score is
 Grammar: 6/10
Fluency: 7/10
Clarity: 7/10
Professional Tone: 8/10
Overall: 7/10


### Simran


Transcribing audio with Whisper...
{'frame_rate': 48000, 'n_frames': 2974936, 'duration': 61.977833333333336, 'average_loudness': 1086, 'silence_ratio': 0.3843765042340407, 'speech_detected': True}
Analysis Results:
Pitch Mean: 1110.78, Pitch Std: 1022.18
Speech Rate: 2.32 words per second
Number of Pauses: 10
Filler Count: 0
Volume Mean: 0.03, Volume Std: 0.02

Your final Audio Score is 33.02773004044332
Your Audio Text score is
 Grammar: 6/10
Fluency: 7/10
Clarity: 8/10
Professional Tone: 6/10
Overall: 7/10

### Diksha
Transcribing audio with Whisper...
{'frame_rate': 48000, 'n_frames': 1918976, 'duration': 39.97866666666667, 'average_loudness': 2163, 'silence_ratio': 0.3077491589264274, 'speech_detected': True}
Analysis Results:
Pitch Mean: 1012.19, Pitch Std: 874.93
Speech Rate: 2.50 words per second
Number of Pauses: 4
Filler Count: 0
Volume Mean: 0.05, Volume Std: 0.03

Your final Audio Score is 54.47535331831925
Your Audio Text score is
 Grammar: 4/10
Fluency: 3/10
Clarity: 3/10
Professional Tone: 4/10
Overall: 3.5/10

### UPSE
Transcribing audio with Whisper...
{'frame_rate': 48000, 'n_frames': 3510528, 'duration': 73.136, 'average_loudness': 1816, 'silence_ratio': 0.3303266631116459, 'speech_detected': True}
Analysis Results:
Pitch Mean: 1090.25, Pitch Std: 1042.80
Speech Rate: 3.19 words per second
Number of Pauses: 17
Filler Count: 0
Volume Mean: 0.04, Volume Std: 0.04
Your final Audio Score is 36.374594931337164

### Fraction1
{'frame_rate': 44100, 'n_frames': 3376128, 'duration': 76.55619047619048, 'average_loudness': 747, 'silence_ratio': 0.4308845073409539, 'speech_detected': True}
Analysis Results:
Pitch Mean: 1004.19, Pitch Std: 1139.31
Speech Rate: 2.47 words per second
Number of Pauses: 22
Filler Count: 0
Volume Mean: 0.02, Volume Std: 0.01
Your final Audio Score is 12.316524498962366

### Fraction2
Transcribing audio with Whisper...
{'frame_rate': 44100, 'n_frames': 3598336, 'duration': 81.59492063492064, 'average_loudness': 1304, 'silence_ratio': 0.3922793758003699, 'speech_detected': True}
Analysis Results:
Pitch Mean: 1090.03, Pitch Std: 1072.64
Speech Rate: 2.19 words per second
Number of Pauses: 15
Filler Count: 0
Volume Mean: 0.02, Volume Std: 0.02
Your final Audio Score is 30.54754294871469-


### Upse2
{'frame_rate': 48000, 'n_frames': 1532064, 'duration': 31.918, 'average_loudness': 2869, 'silence_ratio': 0.27840090231217496, 'speech_detected': True}
Analysis Results:
Pitch Mean: 954.44, Pitch Std: 864.69
Speech Rate: 3.01 words per second
Number of Pauses: 9
Filler Count: 0
Volume Mean: 0.07, Volume Std: 0.06
Your final Audio Score is 58.56900879745011

### Rahul_new
{'frame_rate': 48000, 'n_frames': 2856856, 'duration': 59.517833333333336, 'average_loudness': 1450, 'silence_ratio': 0.38694809958919874, 'speech_detected': True}
Analysis Results:
Pitch Mean: 1174.07, Pitch Std: 971.45
Speech Rate: 2.15 words per second
Number of Pauses: 18
Filler Count: 0
Volume Mean: 0.03, Volume Std: 0.03
Transcript:  I am Rahul Dharmare, a graphic designer with a passion for creativity. I studied graphic design at Zika Indore and also have a Bachelor of Commerce degree from Vikram University. Along the way, I have learned photo editing and video editing and like app designing and website designing. Apart from designing, I also sketch. I am also a sketch artist. I love drawing and bringing ideas to life on paper. Whether it's creating a logo design, a website or sketching something just for fun. I enjoy every part of the creative process. For me, design isn't just a work, it's something I truly love doing. I am always exploring new styles, learning new things and pushing myself to get better every day excited to keep growing and creating.
Your final Audio Score is 28.876778183296064

### Atul_new_fast
Transcribing audio with Whisper...
{'frame_rate': 48000, 'n_frames': 4526208, 'duration': 94.296, 'average_loudness': 3428, 'silence_ratio': 0.24767774702355702, 'speech_detected': True}
Analysis Results:
Pitch Mean: 1149.67, Pitch Std: 999.04
Speech Rate: 2.45 words per second
Number of Pauses: 6
Filler Count: 1
Volume Mean: 0.08, Volume Std: 0.06
Your final Audio Score is 48.37588409889537

### aleena_youtube
Transcribing audio with Whisper...
{'frame_rate': 48000, 'n_frames': 34468463, 'duration': 718.0929791666666, 'average_loudness': 4481, 'silence_ratio': 0.21236474919116644, 'speech_detected': True}
Analysis Results:
Pitch Mean: 1726.77, Pitch Std: 1099.83
Speech Rate: 2.74 words per second
Number of Pauses: 45
Filler Count: 4
Volume Mean: 0.11, Volume Std: 0.09
Your final Audio Score is 12.427740665212433,  55.319


### Elevenlab_new
{'frame_rate': 44100, 'n_frames': 6141312, 'duration': 139.2587755102041, 'average_loudness': 882, 'silence_ratio': 0.43401019195898205, 'speech_detected': True}
Analysis Results:
Pitch Mean: 1656.85, Pitch Std: 1063.14
Speech Rate: 2.03 words per second
Number of Pauses: 58
Filler Count: 0
Volume Mean: 0.02, Volume Std: 0.02
Average
0.19768812452411813
 0.15585025479897455
Your final Audio Score is 10.459467431285896

In [ ]:
# try:
#   number = int(input('Enter your number'))
# except :
#   print('this is not valid')

## Background noise and Breathe detection

In [ ]:
pip install noisereduce

In [ ]:
import IPython
from scipy.io import wavfile
import noisereduce as nr
import soundfile as sf
from noisereduce.generate_noise import band_limited_noise
import matplotlib.pyplot as plt
import urllib.request
import numpy as np
import io

In [ ]:
url = "/content/hema_new.wav"
# response = urllib.request.urlopen(url)
data, rate = sf.read(io.BytesIO(url.read()))
data = data

AttributeError: 'str' object has no attribute 'read'

In [ ]:
url = '/content/hema_new.wav'

In [ ]:
IPython.display.Audio(url, rate=rate)

NameError: name 'rate' is not defined

### Text information Extraction  from the audio text

In [ ]:
import whisper
import warnings

whisper_model = whisper.load_model("medium")
#Audio to text using Whisper
def transcribe_audio_whisper(file_path):
    warnings.filterwarnings("ignore", message="You are using `torch.load` with `weights_only=False`")
    print("Transcribing audio with Whisper...")
    result = whisper_model.transcribe(file_path, fp16=False)
    return result["text"]


text = transcribe_audio_whisper(file_path='/content/atul_new_slow.wav')


import os
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain_google_genai import ChatGoogleGenerativeAI



llm = ChatGoogleGenerativeAI(model='gemini-1.5-pro-latest', temperature=0.6, google_api_key=google_api_key)

memory = ConversationBufferMemory()
conversation_chain = ConversationChain(llm=llm, memory=memory)


analysis_prompt = f""" {text} Analysis the text this is the text of a self introduction of candidate Identify important information from this text like candidate name candidate experience and other details that are important correct details if its seems incorrect like degree name experience skills name

"""


def chat_with_bot(user_input):
    # Format the analysis prompt with user input
    formatted_prompt = analysis_prompt.format(text=user_input)
    response = conversation_chain.run(input=formatted_prompt)
    return response

print(f"Your Audio Text score is \n {chat_with_bot(user_input=analysis_prompt)}")




100%|█████████████████████████████████████| 1.42G/1.42G [00:26<00:00, 58.6MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

Transcribing audio with Whisper...


<ipython-input-4-964deb66aa88>:29: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()
<ipython-input-4-964deb66aa88>:30: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation_chain = ConversationChain(llm=llm, memory=memory)
<ipython-input-4-964deb66aa88>:41: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = conversation_chain.run(input=formatted_prompt)


Your Audio Text score is 
 Atul Kumar Patel's self-introduction reveals the following information:

**Personal Information:**

* **Name:** Atul Kumar Patel

**Education:**

* **Bachelor of Science:**  UPS APS University, Rewa (It's likely "Rewa," not "Riva")
* **Master's in Geology:** Holkar Science College, Indore
* **Post Graduate Diploma (PGD):**  M. Chaturvedi University, Bhopal.  (The specific subject of the PGD isn't mentioned, which might be helpful context.)
* **Data Analytics Certification:** Volcanos Academy, Indore

**Experience:**

* **Current Role:** Data Analyst at VSCO Group, Indore. (While VSCO is a known photo/video editing app company, having an office in Indore seems unusual.  It's possible this is a different company with the same acronym or a typo.)
* **Responsibilities:**  Gathering, processing, and analyzing large datasets to support data-driven decision-making. Creating data visualizations and reports.

**Skills:**

* **Tools:** Power BI, Tableau, Excel, SQL
* *

## Whisper small model
 My name is Sathur Kumar Bhattel. I am a dedicated and result driven data analyst with a strong background in data analysis, business intelligence and machine learning. My academic journey began with a bachelor of science and from UP. APS University, Riva followed by master's degree in geology from Holker Science College in the world. It expanded my technical knowledge. I also completed a PGDC from Markanlal Chaturvedi University, Bhopal and certified data analytics course from Volcanus Academy in Dore. Currently, I work at VSCO Group in Dore as a data analyst where I specialize in a gathering process and analyze large data set to help company and data driven decision. I have hands-on expertise in a tool like Power BI, Tableau, Excel and SQL to data visualizations and reporting along with Python and statistics for deeper analysis and model building. In my free time, I love playing cricket, listening music and exploring new places. I enjoy solving complex problems using data and take pride in presenting inside that drive business growth. My goal is to continue to learn and apply my knowledge to help companies stay ahead of the curve in today's data driven world.

### Whisper medium model
 My name is Atul Kumar Patel. I am a dedicated and result-driven data analyst with a strong background in data analysis, business intelligence and machine learning. My academic journey began with a Bachelor of Science and from UPS APS University, Riva, followed by a Master's degree in Geology from Holkar Science College, Indore. It expanded my technical knowledge. I also completed PGD from M. Chaturvedi University, Bhopal and certified data analytics course from Volcanos Academy, Indore. Currently, I work at VSCO Group, Indore as a data analyst where I specialize in gathering process and analyze large data sets to help company and data-driven decision. I have hands-on expertise in a tool like Power BI, Tableau, Excel and SQL to data visualizations and reporting along with Python and statistics for deeper analysis and model building. In my free time, I love playing cricket, listening music and exploring new places. I enjoy solving complex problems using data and take pride in presenting insights that drive business growth. My goal is to continue to learn and apply my knowledge to help companies stay ahead of the curve in today's data-driven world.

In [ ]:
print(text)

 My name is Atul Kumar Patel. I am a dedicated and result-driven data analyst with a strong background in data analysis, business intelligence and machine learning. My academic journey began with a Bachelor of Science and from UPS APS University, Riva, followed by a Master's degree in Geology from Holkar Science College, Indore. It expanded my technical knowledge. I also completed PGD from M. Chaturvedi University, Bhopal and certified data analytics course from Volcanos Academy, Indore. Currently, I work at VSCO Group, Indore as a data analyst where I specialize in gathering process and analyze large data sets to help company and data-driven decision. I have hands-on expertise in a tool like Power BI, Tableau, Excel and SQL to data visualizations and reporting along with Python and statistics for deeper analysis and model building. In my free time, I love playing cricket, listening music and exploring new places. I enjoy solving complex problems using data and take pride in presenting

In [ ]:
import pandas as pd

In [ ]:
data=[{
    "name": "litik",
    "designation": "AI",
    "salary": 60000,
    "location": "Italy"
  },
  {
    "name": "rulali",
    "designation": "Backend",
    "salary": 12000,
    "location": "India"
  },
  {
    "name": "lulul",
    "designation": "Analysis",
    "salary": 80000,
    "location": "UAE"
  },
  {
    "name": "lakacha",
    "designation": "Backend",
    "salary": 50000,
    "location": "Germany"
  }]

df=pd.DataFrame(data)

In [ ]:
df

,name,designation,salary,location
0,litik,AI,60000,Italy
1,rulali,Backend,12000,India
2,lulul,Analysis,80000,UAE
3,lakacha,Backend,50000,Germany


In [ ]:
for i in df:
  print(i)

name
designation
salary
location


In [ ]:
# df[df['designation','salary']]=='Backend',50000

In [ ]:
df[df['salary']==50000]

,name,designation,salary,location
3,lakacha,Backend,50000,Germany


In [ ]:
df[df['designation']=='Backend']


,name,designation,salary,location
1,rulali,Backend,12000,India
3,lakacha,Backend,50000,Germany


In [ ]:
if

In [ ]:
df[df['designation']=='Backend']

In [ ]:
# df.loc['designation']=='Backend'

In [ ]:
k = 'ritik bhaws from indore'

In [ ]:
len(k.split())

4

In [ ]:
var = k.split()
inter = []
for i in range(len(k.split())):
  inter.append(len(var[i]))

print(inter.index(max(inter)))
print(var[inter.index(max(inter))])




3
indore
